# GPU 설정

In [1]:
import torch

device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

In [2]:
print (f"PyTorch version:{torch.__version__}")
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}")
!python -c 'import platform;print(platform.platform())'

PyTorch version:2.4.0
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-14.6.1-arm64-arm-64bit


# 선언

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import matplotlib as plt
import numpy as np
import pandas as pd

# import Data_Precleaning

# 데이터 확인

In [ ]:
def plot_history(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

# 모델 정의

In [ ]:
class NoiseCNN(nn.Module):
    def __init__(self, input_dim, n_classes):
        super(NoiseCNN, self).__init__()
        self.cnn_model(input_dim, n_classes)
        self.criterion = nn.CrossEntropyLoss()

    def cnn_model(self, input_dim, n_classes):
        self.conv1 = nn.Conv2d(1, 32, kernel_size = (4, 10), padding = (1, 4))
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size = (4, 10), padding = (1, 4))
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size = (4, 10), padding = (1, 4))
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, kernel_size = (4, 10), padding = (1, 4))
        self.bn4 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.flatten = nn.Flatten()

        output_size = (input_dim[0] // 16) * (input_dim[1] // 16) * 32
        
        self.fc1 = nn.Linear(output_size, 64)
        self.bn_fc1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.flatten(x)
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.fc2(x)
        return x
    
    def backward(self, dataloader, optimizer):
        self.train()
        running_loss = 0.0
        for i, (inputs, targets) in enumerate(dataloader):
            optimizer.zero_grad()
            outputs = self(inputs)
            loss = self.criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 10 == 9:
                print(f"Batch {i + 1}, Loss : {running_loss / 10:.4f}")
                running_loss = 0.0
    
    def fit(self, dataloader, epochs, learning_rate):
        optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")
            self.backward(dataloader, optimizer)
            print("")

    def test_eval(self, dataloader):
        self.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        all_preds = []
        all_targets = []

        with torch.no_grad():
            for inputs, targets in dataloader:
                outputs = self(inputs)
                loss = self.criterion(outputs, targets)
                total_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
        
        accuracy = correct / total
        avg_loss = total_loss / len(dataloader)

        print(f"Test Loss : {avg_loss:.4f}, Accuracy : {accuracy * 100:.2f}%")
        
        return avg_loss, accuracy, all_targets, all_preds

# 파라미터 정의

In [ ]:
val = {
    'input_dim' : (128, 128),
    'n_classes' : 10,
    'learning_rate' : 0.01,
    'epochs' : 100,
    'batch_size' : 16
}

# 데이터 학습

## 모델 인스턴스 생성

In [ ]:
model = NoiseCNN(val['input_dim'], val['n_classes'])

In [ ]:
model.summary()

## 데이터 불러오기

In [ ]:
dataset = 
# dataloader = DataLoader(dataset, val['batch_size'], suffle = True)

# 학습

In [ ]:
history = model.fit(dataloader, val['epochs'], val['learning_rate'])

# 결과 확인

In [ ]:
plot_history(history)

# 테스트

In [ ]:
test_dataset = 
# test_dataloader = DataLoader(test_dataset, val['batch_size'], shuffle = False)

In [ ]:
test_history = model.test_eval(test_dataloader)

# 비교 분석

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
def loss_accuracy(avg_loss, accuracy):
    """손실과 정확도 그래프"""
    plt.figure(figsize=(12, 5))
    
    # 손실
    plt.subplot(1, 2, 1)
    plt.bar(['Test Loss'], [avg_loss], color='blue')
    plt.title('Test Loss')
    plt.ylabel('Loss')

    # 정확도
    plt.subplot(1, 2, 2)
    plt.bar(['Test Accuracy'], [accuracy], color='green')
    plt.title('Test Accuracy')
    plt.ylabel('Accuracy')
    
    plt.show()

def confusion_matrix_and_classification_report(targets, preds):
    """혼동 행렬 및 분류 보고서"""
    cm = confusion_matrix(targets, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
    
    print("Classification Report:")
    print(classification_report(targets, preds))

def model_performance(avg_loss, accuracy, targets, preds):
    """모델 성능 평가"""
    loss_accuracy(avg_loss, accuracy)
    confusion_matrix_and_classification_report(targets, preds)

In [ ]:
avg_loss, accuracy, targets, preds = model.test_eval(test_dataloader)

model_performance(avg_loss, accuracy, targets, preds)